In [ ]:
 # import libraries
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-12-06 12:58:36--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   137MB/s    in 0.2s    

2023-12-06 12:58:37 (137 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
df_books

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato


In [ ]:
df_ratings['rating'].value_counts()

0.0     716109
8.0     103736
10.0     78610
7.0      76457
9.0      67541
5.0      50974
6.0      36924
4.0       8904
3.0       5996
2.0       2759
1.0       1770
Name: rating, dtype: int64

In [ ]:
# add your code here - consider creating a new cell for each section of code
df_books[df_books['title']=="I'll Be Seeing You"]

,isbn,title,author
45,0671888587,I'll Be Seeing You,Mary Higgins Clark
74047,0553567187,I'll Be Seeing You,LURLENE MCDANIEL


In [ ]:
df_ratings[df_ratings['isbn']=='0671888587']

,user,isbn,rating
2158,277478,0671888587,0.0
3644,278137,0671888587,0.0
9617,39,0671888587,7.0
10729,487,0671888587,6.0
33292,7904,0671888587,0.0
...,...,...,...
1117151,268032,0671888587,8.0
1129862,271195,0671888587,0.0
1131229,271284,0671888587,0.0
1137934,273979,0671888587,0.0


In [ ]:
cont_user = df_ratings.user.value_counts()
cont_user

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: user, Length: 105283, dtype: int64

In [ ]:
cont_books = df_ratings.isbn.value_counts()
cont_books

0971880107     2502
0316666343     1295
0385504209      883
0060928336      732
0312195516      723
               ... 
1568656386        1
1568656408        1
1569551553        1
1570081808        1
05162443314       1
Name: isbn, Length: 340556, dtype: int64

In [ ]:
#Get the values that have at least 200 user instances and 100 book instances.
user200 = cont_user[cont_user >= 200].index
bokks100 = cont_books[cont_books >= 100].index
user= df_ratings[df_ratings['user'].isin(user200)].copy()
bokks = df_ratings[df_ratings['isbn'].isin(bokks100)].copy()
df_ratings = df_ratings[df_ratings['user'].isin(user200) & df_ratings['isbn'].isin(bokks100)].copy()
df = df_ratings['rating']
# df

In [ ]:
#removes scores from books that are not in the book dataframe
df_ratings = df_ratings[df_ratings['isbn'].isin(df_books['isbn'])]
df_ratings.reset_index(inplace=True, drop=True)
df_ratings

,user,isbn,rating
0,277427,002542730X,10.0
1,277427,0060930535,0.0
2,277427,0060934417,0.0
3,277427,0061009059,9.0
4,277427,0140067477,0.0
...,...,...,...
49512,275970,0804111359,0.0
49513,275970,140003065X,0.0
49514,275970,1400031346,0.0
49515,275970,1400031354,0.0


In [ ]:

#as ratings are in range 1-10, remove all ratings = 0
df_ratings = df_ratings[df_ratings['rating']!=0]
df_ratings.reset_index(inplace=True, drop=True)
df_ratings['rating'].unique()

array([10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  1.,  2.], dtype=float32)

In [ ]:
# df_rat =df_ratings['user'].isin(user200)
# df_ratings[df_rat]

In [ ]:
#min(bokks.isbn.value_counts())

In [ ]:
#min(df.user.value_counts())

In [ ]:
#min(df.isbn.value_counts())

In [ ]:
df_books[df_books['title']=="Where the Heart Is (Oprah's Book Club (Paperback))"]

,isbn,title,author
706,0446672211,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts


In [ ]:
df_ratings[df_ratings['isbn'] == '0140120831']

,user,isbn,rating


In [ ]:
df_books[df_books['isbn'] == '0140120831']

,isbn,title,author


In [ ]:
#df[df['isbn']=='0345351525']

In [ ]:
#samples = df['rating'].to_list()
#samples.reshape(-1, 1)
#lista = np.array(samples).reshape(1, -1)
#len(samples)
#lista

In [ ]:
# neigh = NearestNeighbors(n_neighbors=6, radius=0.4)
# neigh.fit(lista)

In [ ]:
#print(neigh.kneighbors([[1., 1., 1.]]))

In [ ]:
# isbn = df_books['isbn'][df_books['title']=="Where the Heart Is (Oprah's Book Club (Paperback))"].values
# isbn[0]

In [ ]:
# book_rating = df[df['isbn']==isbn[0]]
# book_rating = book_rating['rating'].to_list()
# print(book_rating)
# lista = np.array(book_rating).reshape(-1, 1)
# #print(lista)
# neigh = NearestNeighbors(n_neighbors=5, radius=0.4)
# neigh.fit(lista)
# lista

In [ ]:
#rng = neigh.radius_neighbors([[8.]])
#rng

In [ ]:
#distances, neighbor_indices = neigh.kneighbors(lista)

In [ ]:
# Obtener los libros recomendados basados en los vecinos más cercanos
# recommended_books = []
# listo = []
# for idx, distance in zip(neighbor_indices[0], distances[0]):
#         listo.append([df_books.loc[idx, 'title'], distance])

# recommended_books.append("Where the Heart Is (Oprah's Book Club (Paperback))")
# # for idx, distance in zip(neighbor_indices[0][1:], distances[0][1:]):
# #         listo.append((df_books.loc[idx, 'title'], distance))

# recommended_books = [recommended_books, listo]
# recommended_books


In [ ]:
# book_rating = df[df['isbn']==isbn[0]]
# book_rating

In [ ]:
df_ratings[df_ratings['rating']==0]

,user,isbn,rating


In [ ]:
#add title and author to rating dataframe
correspondencias = dict(zip(df_books['isbn'], df_books['title']))
df_ratings['title'] = df_ratings['isbn'].map(correspondencias)
correspondencias = dict(zip(df_books['isbn'], df_books['author']))
df_ratings['author'] = df_ratings['isbn'].map(correspondencias)

<ipython-input-35-c141d5090185>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ratings['title'] = df_ratings['isbn'].map(correspondencias)
<ipython-input-35-c141d5090185>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ratings['author'] = df_ratings['isbn'].map(correspondencias)


In [ ]:
#transform dataframe to a pivot table for a rigth representation of data
df_promedios = df_ratings.pivot_table(index='title', columns='user', values='rating').fillna(0)

In [ ]:
df_promedios

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,273979,274004,274061,274301,274308,274808,275970,277427,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# function to return recommended books - this will be tested
# def get_recommends(book = ""):
#   recommended_books = [book]
#   #print(book)
#   listo = []
#   #isbn = df_books['isbn'][df_books['title']==book].values
#   #book_rating = df_promedios[df_promedios['title']==book]
#   #book_rating = book_rating['rating'].values
#   #lista = np.array(book_rating).reshape(-1, 1)
#   #suma_elementos = sum(lista)
#   #promedio = suma_elementos / len(lista)
#   #print(promedio[0])
#   #todas las notas
#   #books_rating = df_promedios['rating'].values
#   #books_rating = np.array(books_rating).reshape(-1, 1)

#   print('tamaño de los datos, : ', len(df_promedios))
#   neigh = NearestNeighbors(n_neighbors=5, radius=1)
#   neigh.fit(df_promedios)
#   #A = neigh.kneighbors_graph([promedio])
#   #A.toarray()
#   #plt.plot(A)
#   #print(A)
#   distances, neighbor_indices=neigh.kneighbors(df_promedios.loc[[book]], 6)
#   #distances, neighbor_indices = neigh.kneighbors(lista, 5)
#   print("Recommended Books:")
#   print("==================")
#   for index, value in enumerate(df_promedios.index[neighbor_indices][0][1:]):
#     print((index+1),". ",value)
#     listo.append([value, distances[0][index+1]])
#   del listo[-1]
#   print(neighbor_indices)
#   print(distances)
#   print("Recommended Books:", neighbor_indices[0][1:])

#   # for value, distance in zip(df_promedios.index[neighbor_indices][0][1:], distances[0][1:]):
#   #        listo.append([value, distance])
#   recommended_books = [recommended_books, listo]
#   #neigh.kneighbors([[1., 1., 1.]])
#   return recommended_books

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = [book]
  listo = []

  neigh = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=10, radius=0.5)
  neigh.fit(df_promedios)

  distances, neighbor_indices=neigh.kneighbors(df_promedios.loc[[book]], 7)
  #distances, neighbor_indices = neigh.kneighbors(lista, 5)
  # print('Distances: ', distances)
  # print('neighbor_indices: ', neighbor_indices)
  # print("Recommended Books:")
  # print("==================")
  index_array = np.array(df_promedios.index)
  for index, value in enumerate(index_array[neighbor_indices][0][2:]):
    #print((index+1),". ",value)
    listo.append([value, distances[0][index+2]])
  del listo[-1]
  #print(neighbor_indices)
  #print(distances)

  recommended_books = [book, listo]
  #neigh.kneighbors([[1., 1., 1.]])
  return recommended_books

In [ ]:
#Here code is tested
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['I Know This Much Is True', 0.76770747], ['The Surgeon', 0.7699411], ['The Weight of Water', 0.77085835], ["I'll Be Seeing You", 0.8016211]]]
You passed the challenge! 🎉🎉🎉🎉🎉
